# 절차

- 시퀀스 준비 -> add() -> compile() -> fit() -> 평가

In [1]:
import keras
from keras import models, layers, backend
from keras.datasets import mnist
%tensorflow_version 1.x

Using TensorFlow backend.


# 데이터 확보 -> 준비

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
X_train.shape, y_train.shape
# y_train, y_test는 벡터화 처리 해야 한다.

((60000, 28, 28), (60000,))

In [4]:
# 이미지 크기
img_rows, img_cols = X_train.shape[1:]

img_rows, img_cols

(28, 28)

In [5]:
# 채널을 설명하는 값에 의해 데이터에 채널값을 앞 혹은 뒤에 추가하여 demension(차원)을 확장한다.
# 텐서플로우에서는 안보였던 부분
backend.image_data_format()

'channels_last'

In [0]:
# 입력 데이터 보정(reshape) -> 4D
if backend.image_data_format() == 'channels_first':
    
    # [60000, 28, 28] -> [60000, 1, 28, 28]
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)

else:

    # [60000, 28, 28] -> [60000, 28, 28, 1] <- 텐서플로우의 CNN 예제의 x의 shape 이었다.
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [7]:
X_train[0][0]   # dtype=unit8 -> float32로 변환

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=uint8)

In [0]:
# 배열의 형변환
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
# 데이터의 정규화, ~/256(총 분류 개수로 정규화), ~/255(최대값으로 정규화)
X_train /= 255
X_test /= 255

In [10]:
# 레이블 데이터 -> 벡터화
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [11]:
# 10: 0~9까지 구분되는 수의 개수
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

y_train.shape, y_train[0]

((60000, 10), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

# CNN 레이어 설계

- 시퀀스 -> add() -> compile() -> fit() -> 평가

In [12]:
# 모델준비(시퀀스)
model = models.Sequential()

In [13]:
# 합성곱층 추가 : 채널수 증가, 특징을 추출
# filters : 32 -> 출력 채널수
# kernel_size : 가중치(w) shape
# strides : (1, 1) -> 커널을 1칸씩 이동시키겠다.(생략해도 1, 1은 동일)
# padding : 보정, 잘라진 이미지의 크기를 보정하기 위해서 행렬을 늘려서 shape을 맞춤
# activation : 활성화 함수
# input_shape : 입력 데이터의 shape
model.add(layers.Conv2D(32,
                        kernel_size=(3, 3),
                        strides=(1, 1),
                        padding='valid',
                        activation='relu',
                        input_shape=input_shape))

In [14]:
# 풀링층 추가 : 이미지 사이즈가 감소, 특징을 강화
model.add(layers.MaxPool2D(pool_size=(2, 2), 
                           strides=None     # 기본값 적용
                           ))

In [15]:
# 과적합 방지 : 데이터에 적용되는 것을 방지, 학습을 방해(부분적으로)
# 0.25 : 설정값
model.add(layers.Dropout(0.25))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# 합성곱층 추가 : 채널수 증가, 특징을 추출
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [0]:
# 풀링층 추가 : 이미지 사이즈가 감소, 특징을 강화
model.add(layers.MaxPool2D(pool_size=(2, 2)))

In [0]:
# 과적합 방지 : 데이터에 적용되는 것을 방지, 학습을 방해(부분적으로)
model.add(layers.Dropout(0.25))

In [0]:
# 데이터를 편다. -> flatten 처리
model.add(layers.Flatten())

In [0]:
# 전결합층 : 데이터를 1차로 펴준다.
# layers.Dense() : 일반적인 계층 1개를 의미
# 7 x 7 x 64 -> Flatten -> 128에 수렴
model.add(layers.Dense(128, activation='relu'))

In [0]:
# 과적합 방지 : 데이터에 적용되는 것을 방지, 학습을 방해(부분적으로)
model.add(layers.Dropout(0.5))

In [0]:
# 출력층 : 데이터를 10개(레이블의 분류 개수만큼)로 수렴
model.add(layers.Dense(10, activation='softmax'))

# 학습 및 평가

In [23]:
# 크로스 엔트로피, 경사 하강법
# 손실값을 줄이기 위해 조금씩 이동시키면서 학습을 수행한다.
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='rmsprop', metrics=['accuracy'])

In [0]:
# 총 학습의 횟수 = 훈련 세대 수
epochs = 10

# 1회 학습시 사용하는 데이터의 양
batch_size = 128

In [26]:
model.fit(X_train, y_train,
          batch_size,
          epochs,
          validation_split=0.25)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 45000 samples, validate on 15000 samples
Epoch 1/10





45000/45000 [==============================] - 47s 1ms/step - loss: 0.3743 - acc: 0.8829 - val_loss: 0.1120 - val_acc: 0.9640
Epoch 2/10
45000/45000 [==============================] - 45s 1ms/step - loss: 0.1249 - acc: 0.9622 - val_loss: 0.0612 - val_acc: 0.9806
Epoch 3/10
45000/45000 [==============================] - 45s 1000us/step - loss: 0.0928 - acc: 0.9727 - val_loss: 0.0550 - val_acc: 0.9840
Epoch 4/10
45000/45000 [==============================] - 45s 998us/step - loss: 0.0778 - acc: 0.9765 - val_loss: 0.0437 - val_acc: 0.9870
Epoch 5/10
45000/45000 [==============================] - 45s 998us/step - loss: 0.0654 - acc: 0.9803 - val_loss: 0.0408 - val_acc: 0.9871
Epoch 6/10
45000/45000 [==============================] - 46s 1ms/step - loss: 0.0600 - acc: 0.9818 - val_loss: 0.0394 - val_acc: 0.9881
Epoch 7/10
45000/45

In [28]:
# 점수
score = model.evaluate(X_test, y_test)

score

10000/10000 [==============================] - 3s 329us/step


[0.03137485747935716, 0.9894]